In [2]:
# check gpu availability
import torch
torch.cuda.is_available()

True

In [5]:
# load data and split into train and test sets
from load_data import sim_arr
from sklearn.model_selection import train_test_split, KFold

# reshape from batch, height, width, channel, to batch, channel, height, width
sim_arr_transformed = sim_arr.reshape(sim_arr.shape[0], sim_arr.shape[3], sim_arr.shape[1], sim_arr.shape[2])
train_set, test_set = train_test_split(sim_arr_transformed, test_size=0.2, random_state=42)

# convert to tensor
train_set = torch.tensor(train_set, dtype=torch.float32)
test_set = torch.tensor(test_set, dtype=torch.float32)

In [6]:
import optuna
from AE_torch import Autoencoder, Searchspace
def objective(trial):
    # define search space
    search_space = Searchspace(trial, input_dim=3, output_dim=3)
    # define model
    model = Autoencoder(*search_space)
    # define optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=trial.suggest_float("lr", 1e-5, 1e-1))
    # define loss function
    criterion = torch.nn.MSELoss()
    # train model with k-fold cross validation
    kf = KFold(n_splits=5)
    for train_index, val_index in kf.split(train_set):
        train_set_fold = train_set[train_index]
        train_loader = torch.utils.data.DataLoader(train_set_fold, batch_size=32, shuffle=True)
        val_set_fold = train_set[val_index]
        val_loader = torch.utils.data.DataLoader(val_set_fold, batch_size=32, shuffle=False)

        num_epochs = 10
        for epoch in range(num_epochs):
            model.train()
            train_loss = 0
            # train loop
            for batch in train_loader:
                optimizer.zero_grad()
                output = model(batch)
                loss = criterion(output, batch)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
            # validation loop
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for batch in val_loader:
                    output = model(batch)
                    loss = criterion(output, batch)
                    val_loss += loss.item()
            
        train_loss /= len(train_set_fold)
        val_loss /= len(val_set_fold)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss}, Val Loss: {val_loss}")

    return val_loss

# define study
study = optuna.create_study(direction="minimize",
                            pruner=optuna.pruners.HyperbandPruner(),
                            study_name="autoencoder_torch1",
                            storage="sqlite:///autoencoder.db",
                            load_if_exists=True)
study.optimize(objective, n_trials=1)

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", best_params)
print("Best value:", study.best_value)

[I 2024-05-01 13:06:12,435] Using an existing study with name 'autoencoder_torch1' instead of creating a new one.
/home/tux/hyper-param-optim-4-vector-field-clustering/.venv/lib/python3.12/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2, 2, 2, 10] which is of type list.
  warnings.warn(message)
/home/tux/hyper-param-optim-4-vector-field-clustering/.venv/lib/python3.12/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2, 2, 10, 2] which is of type list.
  warnings.warn(message)
/home/tux/hyper-param-optim-4-vector-field-clustering/.venv/lib/python3.12/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage b

TypeError: Object of type type is not JSON serializable